In [1]:
import os
from os import path
from datetime import datetime
import gensim
from gensim import corpora
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler

base_path = '/opt/data/'
ann_path = path.join(base_path, 'pixel-annot-export/pixel_df_list')
map_path = path.join(base_path, 'pixel-ind2ord')
cor_path = path.join(base_path, 'pixel-corpora/pixel_corpus.mm')

In [2]:
class PixelCorpus(object):
    def __init__(self, ann_path, map_path, threshold, norm):
        self.t = threshold
        self.norm = norm
        
        fileinds = [int(os.path.splitext(f)[0]) for f in os.listdir(ann_path)]
        self.fileinds = sorted(fileinds)
        self.ann_path = ann_path
        self.map_path = map_path
        
        #self.filepaths = [path.join(ann_path,'{}.msgpack'.format(f)) for f in sorted(fileinds)]
                
    def __iter__(self):
        if self.norm: 
            scaler = MinMaxScaler(feature_range=(1, 2))
        
        p_ord = 0

        for f_ind in self.fileinds:
            #if (f_ind==30): break
                
            f = path.join(self.ann_path,'{}.msgpack'.format(f_ind))
            o = path.join(self.map_path,'{}.pkl'.format(f_ind))
            p_map = {}
                
            print(str(datetime.now()),f)
            df = pd.read_msgpack(f)
            
            rel_rows = df[df.fdr < self.t]
            
            #print(df.shape[0])
            print(rel_rows.shape[0])
            
            #if self.norm: df[['int']] = scaler.fit_transform(df[['int']])
            if self.norm: rel_rows[['int']] = scaler.fit_transform(rel_rows[['int']])
                
            #for _, pixel_df in df.groupby('p_ind'):
            for _, pixel_df in rel_rows.groupby('p_ind'):
                #rel_rows = pixel_df[pixel_df.fdr < self.t]
                #pixel_doc = list(zip(rel_rows['ion_ind'].tolist(), rel_rows['int'].tolist()))
                pixel_doc = list(zip(pixel_df['ion_ind'].tolist(), pixel_df['int'].tolist()))
                
                p_ind = pixel_df['p_ind'].tolist()[0] 
                p_map[p_ind] = p_ord
                p_ord += 1
                
                yield pixel_doc
            
            with open(o, "wb") as of: pickle.dump(p_map, of) 

In [3]:
threshold = 51
normalize = False

pixel_corpus = PixelCorpus(ann_path, map_path, threshold, normalize)
corpora.MmCorpus.serialize(cor_path, pixel_corpus)

2017-06-28 20:46:32.458980 /opt/data/pixel-annot-export/pixel_df_list/2.msgpack
132762
2017-06-28 20:46:34.019157 /opt/data/pixel-annot-export/pixel_df_list/3.msgpack
238233
2017-06-28 20:46:36.800113 /opt/data/pixel-annot-export/pixel_df_list/4.msgpack
94542
2017-06-28 20:46:38.142256 /opt/data/pixel-annot-export/pixel_df_list/5.msgpack
32241
2017-06-28 20:46:38.453483 /opt/data/pixel-annot-export/pixel_df_list/6.msgpack
4036161
2017-06-28 20:47:04.379760 /opt/data/pixel-annot-export/pixel_df_list/7.msgpack
81523
2017-06-28 20:47:04.840175 /opt/data/pixel-annot-export/pixel_df_list/8.msgpack
126658
2017-06-28 20:47:05.467151 /opt/data/pixel-annot-export/pixel_df_list/9.msgpack
158485
2017-06-28 20:47:06.244419 /opt/data/pixel-annot-export/pixel_df_list/10.msgpack
11749817
2017-06-28 20:48:06.377579 /opt/data/pixel-annot-export/pixel_df_list/11.msgpack
127503
2017-06-28 20:48:07.925919 /opt/data/pixel-annot-export/pixel_df_list/12.msgpack
3054906
2017-06-28 20:48:25.492842 /opt/data/pi